In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbs
import pickle as pkl

log_folder_location = "logs_dict" #where to find the processed logs
sdiss = ['COTN', 'mirror', 'saturate', 'toroidal', 'unif_resample']
# sdiss = ['c', 'm', 's', 't', 'u', 'h']
sdiss_lables = ['COTN', 'Mir', 'Sat', 'Tor', 'Uni']
sdis_label_dict = {k:v for k,v in zip(sdiss, sdiss_lables)}
colors = sbs.color_palette('Set2', len(sdiss)+2)
colors.pop(-2)
# sdiss = ['c', 'm', 's', 't', 'u', 'h']

In [ ]:
sdiss_sorted = ['COTN', 'mirror', 'saturate', 'toroidal', 'unif_resample']

In [ ]:
font = {'size'   : 30}

plt.rc('font', **font)

In [ ]:
%load_ext rpy2.ipython

In [ ]:
from rpy2 import robjects
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

In [ ]:
%%R 
library(data.table)
library(IOHanalyzer)

For the cell below, and any similar ones, the path should be replaced to the path where the corresponding RDS files are located

In [ ]:
%%R -o get_CS_R

get_CS_R <- function(fid, sdis) {
    dsl <- readRDS(paste0("/mnt/c/Users/Diedie/Documents/Repository/DE_Modular/L-SHADE_", sdis, "_CS.rds"))
    ds <- subset(dsl, funcId == fid)
    cs <- ds[[1]]$PAR$by_RT$CS

    return( cs[cs != 0] )
}

In [ ]:
#ECDF version of previous plot 
def create_POIS_ECDF_matrix(algname):
    colors = sbs.color_palette('Set2', len(sdiss)+1)
    colors.pop(-2)
#     add_legend = True
    fig, axes = plt.subplots(nrows=6, ncols=4, sharex='all', sharey='all')
#     plt.figure(figsize=(16,10))
    for idx_0 in range(6):
        for idx_1 in range(4):
            fid = 1 + 4*idx_0 + idx_1
            bins = np.arange(-1.000000000,1.00000001,0.05)
            for sidx, sdis in enumerate(sdiss):
                cos = get_CS_R(fid, sdis)
                # for rep in range(5):
        #             with open(f"{log_folder_location}/{algname}_F{fid}_{5}D_I{1}_{sdis}_R{rep}.pkl", "rb") as f: 
        #                 res = pkl.load(f)
        #             cos += res['cosine']
        # #         dt = get_merged_dt_v3(sdis, F, CR, popsize)
        # #         dt_applied = dt[dt['nr_exceed'] >= 1]
        #         cos = np.array(cos)[~np.isnan(cos)]
                max_len = len(cos)
#                 if add_legend:
#                     axes[idx_0, idx_1].plot(div1s[idx,:], color=colors[sidx], alpha=0.4, label = sdis)
#                     add_legend=False
                axes[idx_0, idx_1].plot([np.sum(cos<=b)/max_len for b in bins], label=f'{sdis_label_dict[sdis]}', lw=2, c=colors[sidx])
#                 plt.plot([np.sum(cos<=b)/max_len for b in bins], label=f'SDIS: {sdis}', lw=2, c=colors[sidx])
                axes[idx_0, idx_1].legend(fontsize=20)
#             plt.ylim(0,1)
#             plt.xlim(0,40)
#             plt.xticks(range(len(bins))[::5],[f"{x:.2f}" for x in bins[::5]])

            if idx_0 == 5:
#                 axes[idx_0, idx_1].set_xlabel(f'CR 0.{CR[1:]}')
                axes[idx_0, idx_1].set_xticks(range(len(bins))[::5])
                axes[idx_0, idx_1].set_xticklabels([f"{x:.2f}" for x in bins[::5]], fontsize=12)

#         axes[idx_0, 0].set_ylabel(f'F: 0.{F[1:]}')
    fig.set_figheight(30)
    fig.set_figwidth(20)
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.03, wspace=0.03)

    plt.savefig(f"Figures/ECDF_BBOB_matrix_{algname}.pdf")

In [ ]:
create_POIS_ECDF_matrix('L-SHADE')

In [ ]:
%%R -o get_POIS_R

get_POIS_R <- function(sdis) {
    dsl <- readRDS(paste0("/mnt/c/Users/Diedie/Documents/Repository/DE_Modular/L-SHADE_", sdis, "_CS.rds"))
    fracs <- lapply(seq(24), function(fid) {
        ds <- subset(dsl, funcId == fid)
      cs <- ds[[1]]$PAR$by_RT$CS

      non_zeros <- cs[cs != 0]
      length(non_zeros) / length(cs)
    })
    fracs
}

In [ ]:
# %%R -o get_CS_R

# get_CS_R <- function(fid, dim, sdis) {
#     dsl <- readRDS(paste0("/mnt/c/Users/Diedie/Documents/Repository/DE_Modular/L-SHADE_", sdis, "_CS.rds"))
#     ds <- subset(dsl, funcId == fid)
#       cs <- ds[[1]]$PAR$by_RT$CS

#     return( cs[cs != 0] )
# }

In [ ]:
%%R -o get_popdiv -o get_corr -o get_fitness

get_popdiv <- function(fid, dim, sdis, version = 'L-SHADE') {
    dsl <- readRDS(paste0("/mnt/c/Users/Diedie/Documents/repository/DE_Modular/", version, "_", sdis, ".rds"))
    ds <- subset(dsl, funcId == fid, DIM == dim)[[1]]
    pdiv <- ds$PAR$by_RT$popdiv

    return( pdiv )
}

get_corr <- function(fid, dim, sdis, version = 'L-SHADE') {
    dsl <- readRDS(paste0("/mnt/c/Users/Diedie/Documents/repository/DE_Modular/", version, "_", sdis, ".rds"))
    ds <- subset(dsl, funcId == fid, DIM == dim)[[1]]
    pdiv <- ds$PAR$by_RT$cumulative_corrected

    return( pdiv )
}

get_fitness <- function(fid, dim, sdis, version = 'L-SHADE') {
    dsl <- readRDS(paste0("/mnt/c/Users/Diedie/Documents/repository/DE_Modular/", version, "_", sdis, ".rds"))
    ds <- subset(dsl, funcId == fid, DIM == dim)[[1]]
    fv <- ds$FV

    return( fv )
}

In [ ]:
def get_pois(fid, dim, sdis):
    corr = get_corr(fid, dim, sdis)
    corr_noncum = corr[1:] - corr[:-1] 
    corr_noncum[corr_noncum < 0] = 0
    fitness = get_fitness(fid, dim, sdis)
    
    pois = 0
    for rep in range(5):
        x = corr_noncum[:,rep]

        solved = np.where(np.array(fitness[:,rep]) < 1e-8)[0]
        
        if len(solved) > 0:
            max_rt = solved[0]
        else:
            max_rt = len(x)
        pois += (np.sum(x[:max_rt]) / max_rt)
    return pois/5

In [ ]:
def plot_final_pois_heatmap(dim):
    pois_array = np.zeros((24,len(sdiss)))
    for sidx, sdis in enumerate(sdiss_sorted):
        for fid in range(1,25):
            pois_array[fid-1, sidx] = get_pois(fid, dim, sdis)
#         temp = get_POIS_R(sdis)
#         pois_array[:, sidx] = np.array(temp).flatten()
    
    plt.figure(figsize=(20,8))
    sbs.heatmap(np.transpose(pois_array), cmap='viridis', vmax=1)
    plt.yticks([i+0.5 for i in range(5)], sdiss_sorted, rotation=45)
    plt.xticks([i+0.5 for i in range(24)], range(1,25))
    plt.xlabel("Function ID")
    plt.ylabel("SDIS")
    plt.tight_layout()
    plt.savefig(f"figures/Final_POIS_{dim}D_I1.pdf")
    plt.show()

In [ ]:
plot_final_pois_heatmap_all_iid(5)

In [ ]:
%%R -o get_POIS_R_30D

get_POIS_R_30D <- function(sdis) {
    dsl <- readRDS(paste0("/mnt/c/Users/diedi/Documents/Repository/DE_Modular/L-SHADE_", sdis, ".rds"))    
    fracs <- lapply(seq(24), function(fid) {
        ds <- subset(dsl, funcId == fid, DIM == 30)[[1]]
          cs <- ds$PAR$by_RT$CS

      non_zeros <- cs[cs != 0]
      length(non_zeros) / length(cs)
    })
    fracs
}

In [ ]:
# def plot_final_pois_heatmap_all_iid(dim):
pois_array = np.zeros((24,len(sdiss)))
for sidx, sdis in enumerate(sdiss_sorted):
    temp = get_POIS_R_30D(sdis)
    pois_array[:, sidx] = np.array(temp).flatten()

plt.figure(figsize=(20,8))
sbs.heatmap(np.transpose(pois_array), cmap='viridis', vmax=1)
plt.yticks([i+0.5 for i in range(5)], sdiss_sorted, rotation=45)
plt.xticks([i+0.5 for i in range(24)], range(1,25))
plt.xlabel("Function ID")
plt.ylabel("SDIS")
plt.tight_layout()
plt.savefig(f"figures/Final_POIS_{30}D_I1.pdf")
plt.show()

In [ ]:
def plot_sliding_window_pois(fid, iid = 1, dim = 5, de_version = 'L-SHADE'):
    # colors = sbs.color_palette('Set2', len(sdiss)+2) #plus 2 here, since 'h' was not used for these experiments
    # colors.pop(-2)
    plt.figure(figsize=(16,10))
    for sidx, sdis in enumerate(sdiss):
        show_label = True
        corr = get_corr(fid, dim, sdis, de_version)
        corr_noncum = corr[1:] - corr[:-1] 
        corr_noncum[corr_noncum < 0] = 0
        
        lambda_ = 18*dim
        initial_lambda_ = lambda_
        used_budget = 0
        budget = dim*10000
        popsize_arr = [lambda_] 
        while used_budget < corr_noncum.shape[0]:
            lambda_ = int(np.round((4 - initial_lambda_)/budget * used_budget + initial_lambda_))
            popsize_arr.append(lambda_) 
            used_budget += lambda_
        
        fitness = get_fitness(fid, dim, sdis, de_version)
        
        for rep in range(5):
            
            x = corr_noncum[:,rep]
            y = popsize_arr
            if show_label:
                plt.plot(range(int(len(y)-10)),[np.sum(x[int(np.sum(y[:i])):int(np.sum(y[:i+10]))])/np.sum(y[i:i+10]) for i in range(len(y)-10)], label=f'{sdis_label_dict[sdis]}', c=colors[sidx])
                show_label = False
            else:
                plt.plot(range(int(len(y)-10)),[np.sum(x[int(np.sum(y[:i])):int(np.sum(y[:i+10]))])/np.sum(y[i:i+10]) for i in range(len(y)-10)], c=colors[sidx])
            solved = np.where(np.array(fitness[:,rep]) < 1e-8)[0]
            if len(solved) > 0:
                solved_idx = np.where(solved[0] < np.cumsum(y))[0][0]
                plt.axvline(solved_idx, color = colors[sidx], ls='--')
    # plt.plot(range(len(x2)),[v/c for c,v in enumerate(np.cumsum(x2))])
#     plt.axvline(49, c='k')
#     plt.axvline(99, c='k')
#     plt.xscale('log')
    plt.legend()
    plt.xlabel("Start Generation")
    plt.ylabel("Fraction of Corrections")
#     plt.xlim(45,50000)
    plt.tight_layout()
    plt.savefig(f"figures/{de_version}F{fid}_I{iid}_{dim}D_window_pois.pdf")
    plt.close()
#     plt.show()

In [ ]:
def plot_diversity(fid, iid = 1, dim = 5, de_version = 'L-SHADE'):
    # colors = sbs.color_palette('Set2', len(sdiss)+2)
    # colors.pop(-2)
    plt.figure(figsize=(16,10))
    for sidx, sdis in enumerate(sdiss):
        show_label = True
        
        popdiv = get_popdiv(fid, dim, sdis, de_version)
        
        lambda_ = 18*dim
        initial_lambda_ = lambda_
        used_budget = 0
        budget = dim*10000
        popsize_arr = [lambda_] 
        while used_budget < popdiv.shape[0]:
            lambda_ = int(np.round((4 - initial_lambda_)/budget * used_budget + initial_lambda_))
            popsize_arr.append(lambda_) 
            used_budget += lambda_
        
        fitness = get_fitness(fid, dim, sdis, de_version)
        
        for rep in range(5):
            n_iters = len(popsize_arr)
            y = popsize_arr
            popdivs  = popdiv[:,rep]
                        
            n_iters = np.max(np.where(len(popdivs) > np.cumsum(popsize_arr))[0])
            
            popdivs = [popdivs[x-1] for x in np.cumsum(popsize_arr[:n_iters])]
            if show_label:
                plt.plot(range(n_iters-1),popdivs[1:], label=f'{sdis_label_dict[sdis]}', c=colors[sidx])
                show_label = False
            else:
                plt.plot(range(n_iters-1),popdivs[1:], c=colors[sidx])
            solved = np.where(np.array(fitness[:,rep]) < 1e-8)[0]
            if len(solved) > 0:
                solved_idx = np.where(solved[0] < np.cumsum(y))[0][0]
                plt.axvline(solved_idx-1, color = colors[sidx], ls='--')
    # plt.plot(range(len(x2)),[v/c for c,v in enumerate(np.cumsum(x2))])
#     plt.axvline(49, c='k')
#     plt.axvline(99, c='k')
    plt.yscale('log')
    plt.legend()
    plt.xlabel("Generation")
    plt.ylabel("Population Diversity")
#     plt.xlim(45,50000)
    plt.tight_layout()
    plt.savefig(f"figures/{de_version}_F{fid}_I{iid}_{dim}D_diversity.pdf")
    plt.close()
#     plt.show()

In [ ]:
for fid in range(1,25):
    for dim in [5,30]:
        plot_sliding_window_pois(fid, dim = dim, de_version='SHADE')

In [ ]:
for fid in range(1,25):
    for dim in [5,30]:
        plot_sliding_window_pois(fid, dim = dim, de_version='L-SHADE')

In [ ]:
for fid range(1,25):
    for dim in [5,30]:
        plot_diversity(fid, dim = dim, de_version='SHADE')

In [ ]:
for range(1,25):
    for dim in [5,30]:
        plot_diversity(fid, dim = dim, de_version='L-SHADE')